In [4]:
# Observable Trends

# Trend One - The first most obvious trend is that male consumers donimate this data set and make up the majority of players at 82%. 
# Trend Two - The second trend is that most game consumers fall between the agaes of 15 and 24, the largest group at 41.1% is 20 to 24 year olds and the second largest group is 15 to 19 year olds making up 23.65% of consumers. 
# Trend Three - My last observation is that only two games appear in both lists for the top five most popular and the top five most profitable games, Final Critic and Stormcaller. 

In [5]:
# import dependecies
import csv
import os
import pandas as pd
import numpy as np

In [6]:
# create file paths
items_df = pd.read_csv("HeroesOfPymoli/generated_data/items_complete.csv")
players_df = pd.read_csv("HeroesOfPymoli/generated_data/players_complete.csv")
purchase_df = pd.read_json("HeroesOfPymoli/generated_data/purchase_data.json")

In [7]:
# Total Player Count

# find total players using count
total_players = players_df['SN'].count()

# create new df for total players
d = {'Total Players': [total_players]}
total_players_df = pd.DataFrame(data=d)
total_players_df

,Total Players
0,1163


In [8]:
# Purchasing Analysis (Total)

# Number of Unique Items
unique_items = purchase_df['Item Name'].nunique()

# Average Purchase Price
mean_purchase = purchase_df['Price'].mean()

# Total Number of Purchases
total_purchases = purchase_df['SN'].count()

# Total Revenue
total_revenue = purchase_df['Price'].sum()

# Create dataframe for purchasing analysis

d = {'Number of Unique Items': [unique_items], 'Average Price':[mean_purchase], 
     'Number of Purchases': [total_purchases], 'Total Revenue':[total_revenue]}
purchasing_analysis_df = pd.DataFrame(data=d)

# map format for currency values
purchasing_analysis_df['Average Price'] = purchasing_analysis_df['Average Price'].map('${:,.2f}'.format)
purchasing_analysis_df['Total Revenue'] = purchasing_analysis_df['Total Revenue'].map('${:,.2f}'.format)

purchasing_analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$2.93,780,"$2,286.33"


In [9]:
# Gender Demographics (of Total Players)

# get gender totals
gender_demo_df = players_df['Gender'].value_counts()

# input totals to df
gender_demo_df = pd.DataFrame(data=gender_demo_df)

# change column title
gender_demo_df.rename(columns={'Gender': 'Total Count'}, inplace=True)

# calculate percentage for each gender
percent_gender = gender_demo_df['Total Count']/total_players * 100

# input to df and format
gender_demo_df["Percentage of Players"] = percent_gender
gender_demo_df['Percentage of Players'] = gender_demo_df['Percentage of Players'].map('{:,.2f}%'.format)

gender_demo_df

,Total Count,Percentage of Players
Male,954,82.03%
Female,187,16.08%
Other / Non-Disclosed,22,1.89%


In [10]:
# Purchasing Analysis (Gender)

# number of purchases
purchase_count = purchase_df.groupby('Gender')['Item ID'].count()

# average price
ave_price = purchase_df.groupby('Gender')['Price'].mean()

# total revenue
total_value = purchase_df.groupby('Gender')['Price'].sum()

# normalize totals
norm_totals = total_value / gender_demo_df['Total Count']



# input to df
purchase_gender = pd.DataFrame({"Purchase Count": purchase_count,
                                "Average Purchase Price": ave_price,
                                "Total Purchase Value": total_value,
                               "Normalized Totals": norm_totals
                               })

# format numbers
purchase_gender['Average Purchase Price'] = purchase_gender['Average Purchase Price'].map('${:,.2f}'.format)
purchase_gender['Total Purchase Value'] = purchase_gender['Total Purchase Value'].map('${:,.2f}'.format)
purchase_gender['Normalized Totals'] = purchase_gender['Normalized Totals'].map('${:,.2f}'.format)

purchase_gender


,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,136,$2.82,$382.91,$2.05
Male,633,$2.95,"$1,867.68",$1.96
Other / Non-Disclosed,11,$3.25,$35.74,$1.62


In [11]:
# Age Demographics

# create bins for age groups
bins = [0, 10, 15, 20, 25, 30, 35, 40, 45]
group_labels = [ '<10', '10-14', '15-19', '20-24', '25-29', '30-34','35-39','>40']

# add age group column to table
pd.cut(players_df["Age"], bins, labels=group_labels)

players_df["Age Group"] = pd.cut(players_df["Age"], bins, labels=group_labels)


# create table for age groups 
# group by age
age_group = players_df.groupby(["Age Group"])

# total per group
total_count = age_group['SN'].count()

# percent
percent_age = age_group['SN'].count()/total_players * 100

# input to table 
age_demo = pd.DataFrame({"Percentage of Players": percent_age,
                                "Total Count": total_count})
# format mapping
age_demo['Percentage of Players'] = age_demo['Percentage of Players'].map('{:,.2f}%'.format)

age_demo

,Percentage of Players,Total Count
Age Group,,
<10,5.33%,62
10-14,7.48%,87
15-19,23.65%,275
20-24,41.10%,478
25-29,9.97%,116
30-34,7.57%,88
35-39,4.13%,48
>40,0.77%,9


In [12]:
# Top Five Spenders

# gather data for each column 

# total number of purchases
top_purchase_count = purchase_df.groupby('SN')['Item ID'].count()

# total of purchases per SN
top_total = purchase_df.groupby('SN')['Price'].sum()

# average purchase per SN
top_ave_price = top_total/top_purchase_count

In [13]:
# input to DF
top_five = pd.DataFrame({"Purchase Count": top_purchase_count, "Average Purchase Price": top_ave_price, "Total Purchase Value": top_total})

# sort and limit to top five
top_five_spenders = top_five.sort_values("Total Purchase Value", ascending=False)
top_five_spenders = top_five_spenders.head()

# map formatting
top_five_spenders['Average Purchase Price'] = top_five_spenders['Average Purchase Price'].map('${:,.2f}'.format)
top_five_spenders['Total Purchase Value'] = top_five_spenders['Total Purchase Value'].map('${:,.2f}'.format)

top_five_spenders

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$3.41,$17.06
Saedue76,4,$3.39,$13.56
Mindimnya67,4,$3.18,$12.74
Haellysu29,3,$4.24,$12.73
Eoda93,3,$3.86,$11.58


In [14]:
# Most Popular Items

# gather data for each column 

# total number of purchases
most_pop_count = purchase_df.groupby('Item Name')['Item ID'].count()

# total of purchases per game
most_pop_rev = purchase_df.groupby('Item Name')['Price'].sum()

# price per game
most_pop_price = most_pop_rev/most_pop_count

# input to DF
most_pop = pd.DataFrame({"Purchase Count": most_pop_count, "Item Price": most_pop_price, "Total Purchase Value": most_pop_rev})

# sort by purchase count and limit to top five
most_pop = most_pop.sort_values("Purchase Count", ascending=False)
most_popular_five = most_pop.head()

# map formatting
most_popular_five['Item Price'] = most_popular_five['Item Price'].map('${:,.2f}'.format)
most_popular_five['Total Purchase Value'] = most_popular_five['Total Purchase Value'].map('${:,.2f}'.format)

most_popular_five

/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Purchase Count,Item Price,Total Purchase Value
Item Name,,,
Final Critic,14,$2.76,$38.60
Arcane Gem,11,$2.23,$24.53
"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
Stormcaller,10,$3.46,$34.65
Woeful Adamantite Claymore,9,$1.24,$11.16


In [15]:
# Most Profitable Items

# gather data for each column 

# total number of purchases
most_prof_count = purchase_df.groupby('Item Name')['Item ID'].count()

# total of purchases per game
most_prof_rev = purchase_df.groupby('Item Name')['Price'].sum()

# price per game
most_prof_price = most_prof_rev/most_prof_count

# input to DF
most_prof = pd.DataFrame({"Purchase Count": most_prof_count, "Item Price": most_prof_price, "Total Purchase Value": most_prof_rev})

# sort by total purchase value and limit to top five
most_prof = most_prof.sort_values("Total Purchase Value", ascending=False)
most_profitable_five = most_prof.head()

# map formatting
most_profitable_five['Item Price'] = most_profitable_five['Item Price'].map('${:,.2f}'.format)
most_profitable_five['Total Purchase Value'] = most_profitable_five['Total Purchase Value'].map('${:,.2f}'.format)

most_profitable_five

/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Purchase Count,Item Price,Total Purchase Value
Item Name,,,
Final Critic,14,$2.76,$38.60
Retribution Axe,9,$4.14,$37.26
Stormcaller,10,$3.46,$34.65
Spectral Diamond Doomblade,7,$4.25,$29.75
Orenmir,6,$4.95,$29.70
